In [4]:
import numpy as np
import pandas as pd
import neurokit2 as nk

In [ ]:
# Code pulled from Neurokit2 Gitub page: https://github.com/neuropsychology/NeuroKit

Neurokit2 wraps its EDA processing functionality into a convenience function: eda_process(). This function takes in the EDA signal data, the sampling rate, and a choice of method ("neurokit" or "biosppy") for cleaning the signal. 

In [ ]:
# The inner workings of eda_process:

def eda_process(eda_signal, sampling_rate=1000, method="neurokit"):
    # ...
    # Sanitize input
    eda_signal = signal_sanitize(eda_signal)

    # Series check for non-default index
    if type(eda_signal) is pd.Series and type(eda_signal.index) != pd.RangeIndex:
        eda_signal = eda_signal.reset_index(drop=True)

    # Preprocess
    eda_cleaned = eda_clean(eda_signal, sampling_rate=sampling_rate, method=method)
    eda_decomposed = eda_phasic(eda_cleaned, sampling_rate=sampling_rate)

    # Find peaks
    peak_signal, info = eda_peaks(
        eda_decomposed["EDA_Phasic"].values,
        sampling_rate=sampling_rate,
        method=method,
        amplitude_min=0.1,
    )

    # Store
    signals = pd.DataFrame({"EDA_Raw": eda_signal, "EDA_Clean": eda_cleaned})

    signals = pd.concat([signals, eda_decomposed, peak_signal], axis=1)

    return signals, info

## Let's break this down piece by piece to confirm that it's behaving as intended

### (1) Sanitize Input

In [ ]:
# The first chunk of logic is: Sanitize input
eda_signal = signal_sanitize(eda_signal)

# Here is the function signal_sanitize
def signal_sanitize(signal):
    # Series check for non-default index
    if type(signal) is pd.Series and type(signal.index) != pd.RangeIndex:
        return signal.reset_index(drop=True)

    return signal

In [8]:
# So this is simply resetting the index of the Series object holding the EDA signal to be a monotonically increasing integer ID. 
# It seems that this logic is repeated right after the authors call signal_sanitize, so we found some duplicate code.

### (2) Preprocess

In [ ]:
# Preprocess starts out by calling the function eda_clean(), which takes our signal data, sampling rate, and choice of method which are "neurokit" or "biosppy"
# If we choose the neurokit method, the following function is called:
def _eda_clean_neurokit(eda_signal, sampling_rate=1000):

    # Filtering
    filtered = signal_filter(eda_signal, sampling_rate=sampling_rate, highcut=3, method="butterworth", order=4)

    return filtered

# This function low-pass filters our EDA signal using the Butterworth method (https://en.wikipedia.org/wiki/Butterworth_filter) 

In [10]:
np.array([3]) / (51.2/2)

array([0.1171875])

#PLAN: 
1. Write brief report about EDA and what we do to it to process the signal and why we care
1b. Plot sample signals that we collected in lab

2. Motivate rest of project by focusing on the cleaning aspect of EDA processing
2a low-pass filtering
2b smoothing
2c removing corrupted signal segments and connect with a spline

3. Step through eda_clean pipeline and use techniques learned in class to check that these are happening as expected. Repeat with A, B, C quality signals
3a. DFT to see which frequencies were removed
3b. Convolve output with [-1, 1] kernel (1st derivative) to see if smoothing worked
3c. Use method in that paper to detect corrupted signal segments. If there's time, connect with a spline

4. Discuss findings and implications for our projects in lab. 